# 1.导入引入

In [50]:
import sys
import json
from io import BytesIO, StringIO
import re

sys.path.append("../")

from loader.unstructured_loaders.pdf_loader import PDFLoader
from loader.unstructured_loaders.doc_loader import DocLoader
from loader.unstructured_loaders.table_loader import TableLoader
from loader.unstructured_loaders.ppt_loader import PPTLoader
from textsplitter.document_header_splitter import DocumentHeaderTextSplitter
from utils.common import convert_bytes_to_file, convert_file_type
from utils.llm_util import post_api_for_llm,apost_api_for_llm

In [51]:
document_splitter = DocumentHeaderTextSplitter()

# 2.测试文件解析

In [158]:
doc_path = "/Users/ucdteam/Downloads/test_files/育儿假、独生子女父母护理假（华南区）执行标准.docx"

# 普通表格和跨页短表格
doc_path = "/Users/ucdteam/Downloads/团购文件测试/7.2.1 大客户销售管理规定B0.pdf"

# 跨页长表格
doc_path = "/Users/ucdteam/Downloads/团购文件测试/7.2.2 实物团购流程A1.pdf"

# 跨页长表格，且有合并单元格

doc_path = "/Users/ucdteam/Downloads/团购文件测试/7.2.2.2 福利购下单配送流程.pdf"

# 跨页长表格，且回复文本内容较长
doc_path = "/Users/ucdteam/Downloads/财务文件测试/8.1.1.1 预算考核与评价流程A1.pdf"


# 复杂表格结构
doc_path = "/Users/ucdteam/Downloads/财务文件测试/8.2.1.1.9 资金支付管理规范A2.pdf"

doc_path = "/Users/ucdteam/Downloads/财务文件测试/8.3.1.2.8 专柜商品订金及租生活押金操作指引A1.pdf"


In [159]:
# doc_loader = DocLoader(file_path=doc_path, file_stream=None)

doc_loader = PDFLoader(file_path=doc_path, file_stream=None)

27-06-2024:16:38:32,656 INFO     [converter.py:126] [1/4] Opening document...
27-06-2024:16:38:32,657 INFO     [converter.py:151] [2/4] Analyzing document...
27-06-2024:16:38:33,385 INFO     [converter.py:159] [3/4] Parsing pages...
27-06-2024:16:38:33,385 INFO     [converter.py:165] (1/5) Page 1
27-06-2024:16:38:33,430 INFO     [converter.py:165] (2/5) Page 2
27-06-2024:16:38:33,471 INFO     [converter.py:165] (3/5) Page 3
27-06-2024:16:38:33,515 INFO     [converter.py:165] (4/5) Page 4
27-06-2024:16:38:33,572 INFO     [converter.py:165] (5/5) Page 5
/Users/ucdteam/digital_labor/test/../loader/unstructured_loaders/pdf_loader.py:416: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. S

In [153]:
docs = document_splitter.split_text(blocks=doc_loader.blocks, use_header=True)

In [146]:
# docs[4].page_content

In [160]:
chunks = []
for i, doc in enumerate(docs):
    page_content = doc.page_content
    metadata = doc.metadata
    chunk = {
        "index": i,
        "raw_text": metadata.get("chunk_raw_text"),
        "html": metadata.get("html", ""),
        "metadata": metadata,
    }
    chunks.append(chunk)

In [161]:
chunks

[{'index': 0,
  'raw_text': '收银核对操作指引',
  'html': '<p style="font-size: 1.5rem">收银核对操作指引</p>',
  'metadata': {'html': '<p style="font-size: 1.5rem">收银核对操作指引</p>',
   'chunk_raw_text': '收银核对操作指引'}},
 {'index': 1,
  'raw_text': '1.目的\n明确收银核对各岗位职责、核对范围及质量要求,规范收银核对步骤,确保公司资金安全及账务的及时性、准确性。',
  'html': '<h1 style="font-size:1.0rem">1.目的</h1>\n<p style="font-size: 1.0rem">明确收银核对各岗位职责、核对范围及质量要求,规范收银核对步骤,确保公司资金安全及账务的及时性、准确性。</p>',
  'metadata': {'Heading 1': {'data': '1.目的',
    'html': '<h1 style="font-size:1.0rem">1.目的</h1>'},
   'html': '<h1 style="font-size:1.0rem">1.目的</h1>\n<p style="font-size: 1.0rem">明确收银核对各岗位职责、核对范围及质量要求,规范收银核对步骤,确保公司资金安全及账务的及时性、准确性。</p>',
   'chunk_raw_text': '1.目的\n明确收银核对各岗位职责、核对范围及质量要求,规范收银核对步骤,确保公司资金安全及账务的及时性、准确性。'}},
 {'index': 2,
  'raw_text': '2.适用范围\n\n2.1 流程归属\nL1:【财务管理】——L2:【会计核算】——L3:【销售及应收管理流程】——L5:【收银核对操作指引】\n2.2 业务范围\n适用于天虹数科商业股份有限公司及其子公司各收银平台收银核对及款项到账确认过程。\n',
  'html': '<h1 style="font-size:1.0rem">2.适用范围</h1>\n\n<h2 style="font-size:1.0rem">2.1 流程归属</h2

In [70]:
# for i, chunk in enumerate(chunks):
#     if i > 2:
#         print(chunk['raw_text'])
#         print("-----------" * 20)

In [162]:
markdown_re = re.compile(r"(\|[:]? [-]+ \|)|(\|[:]?[-]+\|)")  # 匹配markdown的份

def check_table(content):
    """校验content中是否有markdown表格"""
    return bool(markdown_re.search(content))

In [176]:
tchunk = ""
count = 0
for i, chunk in enumerate(chunks):
    raw_text = chunk['raw_text']
    
    if not check_table(raw_text):
        continue
    # print(raw_text)
    # print("*" * 100)
    count += 1
    if count > 1:
        tchunk = raw_text
        break
        

In [177]:
tchunk

'4.文件修订记录:\n\n| 版 本   | 修订责 任人    | 修订日期   | 批准人   | 修订内容及理由（若为优化文件，请概述修改要点）                                                                                                                                                                                                                                      |\n|:----------------|:-----------------------|:-----------|:---------|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|\n| A1              | 许桂梅、钟莹莹 | 2023.6.27  | 左幼琼   | 1、删除“资金到账核算流程”相关内容，整合至“收银核对流程”；2、增加了财务RPA 及收银待办推送等描述；3、更新了相应文字描述，“虹领巾”修改为“天虹APP”、”中台系统“修改为”业务中台“等；4、卡券类收银核对增加了从“消费”到“收入”过渡科目描述，账务流转更加清晰、完整。 |\n'

In [173]:
from pprint import pprint
prompt_test = f"""请根据以下公司规定回答问题。如果没有包含相关信息，则回复“没有相关信息”。
{tchunk}
问题：付款方式是支票时，2.提交付款的操作步骤是什么
"""
print(post_api_for_llm(prompt_test))

27-06-2024:16:39:17,273 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


没有相关信息。根据提供的文件修订记录，没有提及关于支票付款方式或提交付款的操作步骤的信息。


In [87]:
# pprint(prompt_test)

In [46]:
prompt_test

'请根据以下公司规定回答问题。如果没有包含相关信息，则回复“没有相关信息”。\n3、职责\n\n| 总部   | TOB 业务发展部                    | 1、负责公司整体TOB 业务客户洞察及研究，负责公司整体 大客户销售管理、优惠政策制定及优化；2、负责制定公司年度、重大节假日大客户销售目标，组织 各区域/事业部对年度目标进行分解并跟进完成情况；3、负责公司整体TOB 业务费用预算；4、负责大客户销售产品信息收集、行业调研、需求确定，提出开发需求并推进功能开发、上线运用，监督运用效果；5、负责公司整体大客户营销方案制定，推广，实施；5、统筹监督管理各区域/事业部开发目录商品供应商的规范 性，负责TOB 业务发展部战略商品及目录商品客户采购需求 的开发、采购、报价、供应商谈判及货源准备；6、负责大客户销售业务拓展过程中所需提供的公司级相关 资质、体系认证及证照办理工作；7、管理、协助各区域/事业部大客户销售的招投标工作；8、负责TOB 业务系统客户信息管理、维护、监督工作。   |\n|:-------|:----------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [108]:
chat_history = [
    {"role": "user","content":prompt_test},
    {"role": "assistant","content": """计算业绩考核结果时，总部财务部、区域/事业部财务部等部门的活动说明如下：

1. 总部财务部依据“年度考核方案”对各业务单元的预算完成情况进行考核并计算公司整体业绩考核结果。
2. 为了确保业绩考核过程公开透明化，各区域/事业部熟悉掌握业绩考核计算逻辑及过程，并监督本区域/事业部预算执行效果，督促其更好地达成预算目标，由各区域/事业部财务部计算本区域/事业部业绩考核结果并与总部财务部计算结果进行核对，确保计算结果准确无误。
3. 具体内容包括：
   - 月度：月度结束后，总部财务部及区域/事业部财务部同步完成月度考核奖金分数的计算与核对。如果预算单位在执行过程中由于经营环境/经营条件/政策法规等发生重大变化，致使预算执行结果产生重大偏差，可提交“财务管理-预算管理-CWAC 预算考核审批申请-月度业绩考核调整事项申请流程”，经公司审批通过后，总部财务部对业绩得分进行相应调整。
   - 季度：每季度结束后，总部财务部及区域/事业部财务部须同步完成超额利润分享的奖金计算及核对。
   - 年度：年度结束后，总部财务部及区域/事业部财务部须同步完成年度超额分享奖金、年度KPI财务数据统计等业绩计算与核对。"""},
]

In [126]:
print(post_api_for_llm(content="董事长、总经理、总会计师审批业绩考核的活动说明是什么",chat_history=chat_history))

27-06-2024:15:16:30,976 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


在您提供的公司规定流程中，并没有直接提及董事长、总经理、总会计师审批业绩考核的具体活动说明。业绩考核通常涉及对员工或部门的绩效进行评估，以确定其是否达到预定的目标和标准，这通常由人力资源部门或财务部门负责，并可能需要高级管理层的审批。如果董事长、总经理、总会计师参与业绩考核的审批，他们可能会负责以下活动：

1. **审查业绩考核标准和指标**：确保这些标准和指标与公司的战略目标和财务目标相一致，同时确保它们是公平、合理和可量化的。

2. **审批业绩考核结果**：审查由人力资源部门或财务部门提交的业绩考核结果，确保评估过程的公正性和准确性，以及结果的合理性。

3. **决定奖励和惩罚**：基于业绩考核结果，决定是否给予员工或部门奖励，如奖金、晋升或额外福利，或者是否需要采取纠正措施，如培训、辅导或纪律处分。

4. **制定改进计划**：如果业绩考核显示存在不足，高级管理层可能需要制定改进计划，以帮助员工或部门提高绩效。

5. **确保合规性**：确保业绩考核过程符合所有相关的法律法规和公司政策，避免任何潜在的法律或道德风险。

然而，具体的活动说明和流程可能因公司而异，需要参考公司的具体规定和政策。如果需要更详细的信息，建议查阅公司的业绩考核政策或直接咨询公司的人力资源部门或财务部门。


In [178]:
doc_path = "/Users/ucdteam/Downloads/test_files/育儿假东南区执行方案（流程版）20220413.docx"

In [179]:
d = DocLoader(file_path=doc_path)

/Users/ucdteam/digital_labor/test/../loader/unstructured_loaders/doc_loader.py:235: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode = stats.mode(line_heights)[0][0]


In [52]:

tchunk = """
|       | 住宿费          |           |   | 补助（含伙食和交通补助） |
|-------|--------------|----------------|------|--------------|
|       | 公司助理总经理及以上成员 | 部门助理总监/资深及以上人员 | 其他人员 |              |
| 北京    | 650          | 550            | 450  | 100          |
| 上海    | 600          | 500            | 400  | 100          |
| 广州、深圳 | 550          | 450            | 350  | 100          |
| 其他城市  | 480          | 400            | 300  | 100          |
"""

tchunk = """
|    |            | 住宿费                   | 住宿费                      | 住宿费   | 补助(含伙食和交通补助)   |
|---:|:-----------|:-------------------------|:----------------------------|:---------|:-------------------------|
|  0 |            | 公司助理总经理及以上成员 | 部门助理总监/资深及以上人员 | 其他人员 | 补助(含伙食和交通补助)   |
|  1 | 北京       | 650                      | 550                         | 450      | 100                      |
|  2 | 上海       | 600                      | 500                         | 400      | 100                      |
|  3 | 广州、深圳 | 550                      | 450                         | 350      | 100                      |
|  4 | 其他城市   | 480                      | 400                         | 300      | 100                      |
"""

q1 = "北京地区公司助理总经理住宿费标准是多少?"
q2 = "南昌地区公司助理总经理住宿费标准是多少?"
q3 = "部门助理总监在江西区域住宿费标准是多少"
q4 = "北京地区住宿费标准是多少"
q5 = "江西区域住宿费标准是多少"
from pprint import pprint

for q in [q1, q2,q3]:
    prompt_test = f"""请根据以下公司规定回答问题。如果没有包含相关信息，则回复“没有相关信息”。
    {tchunk}
    问题：{q}
    """
    answer = post_api_for_llm(prompt_test)
    print(
        "问题\t",q,
        "对应的答案:\t",answer
    )
    print("-" * 50)

08-07-2024:10:13:53,584 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


问题	 北京地区公司助理总经理住宿费标准是多少? 对应的答案:	 北京地区公司助理总经理的住宿费标准是650元。
--------------------------------------------------


08-07-2024:10:13:55,265 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


问题	 南昌地区公司助理总经理住宿费标准是多少? 对应的答案:	 根据表格，南昌属于其他城市类别，因此公司助理总经理的住宿费标准是480。
--------------------------------------------------


08-07-2024:10:13:56,933 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


问题	 部门助理总监在江西区域住宿费标准是多少 对应的答案:	 部门助理总监在江西区域的住宿费标准为400。因为江西属于其他城市类别。
--------------------------------------------------


In [55]:
tchunk = """
|    |            | 住宿费                   | 住宿费                      | 住宿费   | 补助(含伙食和交通补助)   |
|---:|:-----------|:-------------------------|:----------------------------|:---------|:-------------------------|
|  0 |            | 公司助理总经理及以上成员 | 部门助理总监/资深及以上人员 | 其他人员 | 补助(含伙食和交通补助)   |
|  1 | 北京       | 650                      | 550                         | 450      | 100                      |
|  2 | 上海       | 600                      | 500                         | 400      | 100                      |
|  3 | 广州、深圳 | 550                      | 450                         | 350      | 100                      |
|  4 | 其他城市   | 480                      | 400                         | 300      | 100                      |
"""

q5 = "江西住宿费标准是多少"

prompt_test = f"""请根据以下公司规定回答问题。如果没有包含相关信息，则回复“没有相关信息”。
{tchunk}
问题：{q5}
"""
answer = post_api_for_llm(prompt_test)
answer

08-07-2024:10:24:30,139 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


'根据上述表格，对于江西（属于其他城市类别）的住宿费标准如下：\n\n- 公司助理总经理及以上成员：480元\n- 部门助理总监/资深及以上人员：400元\n- 其他人员：300元\n\n补助（含伙食和交通补助）为100元。'

In [38]:
file_name = "天虹出差补助标准制度文档"

header = "住宿费和补助标准"


test_chunk = file_name + "\n" + header + "\n" + tchunk

q = "天虹出差总经理助理的住宿费补助标准是多少?"

In [39]:
test_chunk

'天虹出差补助标准制度文档\n住宿费和补助标准\n\n|       | 住宿费          |   住宿费             |  住宿费    | 补助（含伙食和交通补助） |\n|-------|--------------|----------------|------|--------------|\n|       | 公司助理总经理及以上成员 | 部门助理总监/资深及以上人员 | 其他人员 |              |\n| 北京    | 650          | 550            | 450  | 100          |\n| 上海    | 600          | 500            | 400  | 100          |\n| 广州、深圳 | 550          | 450            | 350  | 100          |\n| 其他城市  | 480          | 400            | 300  | 100          |\n'

In [66]:
tchunk = """育儿假、延长产假及独生子女护理假（华东区）执行方案
1.独生子女父母护理假的定义:独生子女父母年满60周岁后患病住院期间,独生子女每年享受5天的带薪护理假。
2.假期安排:《实施方案》发布之日(2022年2月10日)起,符合政策法规的独生子女员工,其父母年满60周岁后患病住院期间,独生子女员工每年享受5天的带薪护理假,其每年享受的父母护理假可以一次性享受,也可以分次享受。3.需提供的资料:持独生子女父母光荣证或独生子女证、父母身份证件、医疗机构诊断证明、住院证明等资料到相关部门申请,流程通过后方可享受独生子女护理假。注:当年度新入职、离职员工可享受的假期天数,应当按照员工当年已工作时间折算应休未休护理假天数。折算方法为:(当年度在本单位已过日历天数÷365天)×员工当年度应享受的护理假天数-当年度已安排护理假天数。折算后不满一天时,舍零取整。如出现员工多休护理假情况,按事假扣回。当年度未休完的护理假作清零处理,不另行计发加班工资,也不可延长至下一年度补休。4.其他:若员工所在地关于独生子女父母护理假另有政策规定的,以当地政策为准。
审批流程:![](https://dev-assets-api.tianhong.cn/bigdata/digital-employee/attach/img/2024/7/8/GpnLYoRqyGUPLhJlXJiSM1XQ.png)\\n ![](https://dev-assets-api.tianhong.cn/bigdata/digital-employee/attach/img/2024/7/8/GpnLYoRqyGUPLhJlXJiSM1XQ.png)
华东区行政管理部
2022年3月9日
"""

q6 = "华东区育儿假审批流程是什么"
prompt_test = f"""请根据以下公司规定回答问题，如果包含图片链接请原样输出图片链接，不允许修改图片链接内容。如果没有包含相关信息，则回复“没有相关信息”。
{tchunk}
问题：{q6}
"""
answer = post_api_for_llm(prompt_test)
answer

08-07-2024:16:12:40,983 INFO     [_client.py:1026] HTTP Request: POST http://10.66.92.117:8080/v1/chat/completions "HTTP/1.1 200 OK"


'![](https://dev-assets-api.tianhong.cn/bigdata/digital-employee/attach/img/2024/7/8/GpnLYoRqyGUPLhJlXJiSM1XQ.png)  \n![](https://dev-assets-api.tianhong.cn/bigdata/digital-employee/attach/img/2024/7/8/GpnLYoRqyGUPLhJlXJiSM1XQ.png)'

### 测试语义相似度

In [40]:
from db.embedding_utils import embed_texts_api

In [41]:
qe = embed_texts_api(q)

In [42]:
import numpy as np
import torch

qe = torch.from_numpy(np.array(qe))

In [43]:
test_chunk_e = torch.from_numpy(np.array(embed_texts_api(test_chunk)))

In [44]:
qe @ test_chunk_e.T

tensor([[0.8164]], dtype=torch.float64)

In [1]:
tchunk = """
'出差管理制度规范第一条 目的为规范员工出差流程、出差审批及差旅费报销,特制定本制度。第二条 适用范围本制度适用于X X X公司全体员工。第三条 出差审批权责1、各部门主管负责本部门员工出差和出差费用的审批。2、总经理负责对公司各部门主管级以上人员出差和出差费用的审批。3、财务部负责出差备用金申请的审核、差旅费报销的审核。4、员工出差国外或港澳台地区,一律由总经理审批。第四条 出差管理原则1、出差人员必须按照高效、经济、安全便捷原则选择交通工具、路线和时间,反对浪费行为。2、出差必须先获批准或被安排出差,对擅自主张者不予报销差旅费。3、提倡力行节约并对限额控制的差旅费用实行超支自理。4、公务出差兼私人旅行,公司承担公务出差部分的差旅费。5、业务出差人员须在出差结束后5个工作日内提交《出差报告》,并作为领导是否同意报销差旅费的重要依据。第五条 出差审批1、公司各部门负责人都应合理安排外出人员,并考虑完成任务的期限,确定出差地点、路线和时间等相关事宜,填写《因公出差审批表》(附件一),经有审批权限的领导批准同意。2、出差人员凭《因公出差审批表》办理差旅费预借审批。3、出差中途改变路线时,需报直属领导同意,报销审批时另附说明。第六条 差旅费构成列入本制度的差旅费是员工处理公务或公司外送学习培训、外出会议、考察等活动发生的费用,主要包括:1、长途交通费:乘座飞机、轮船、火车、长途客车的费用。2、市内交通费:乘座公共汽车、出租车、地铁、轻轨列车的费用。3、住宿费:异地出差因过夜住宿发生的费用。4、膳食补贴:按地区标准和出差期应就餐次数及补贴限额计发补助。5、其它费用:公务发生的电传、邮寄、托费、公共服务等费用。第七条 出差费报销标准员工出差乘坐车、船、飞机和食宿费用都应力行节约,并按不超出规定限额报销。限额控制的具体标准:全国区域出差标准:\n| 类别             | 城市类别                                            | 住宿费                                              | 餐费                                                | 长途交通费                                          | 市内交通费                                          | 备注                                                |\n|:-----------------|:----------------------------------------------------|:----------------------------------------------------|:----------------------------------------------------|:----------------------------------------------------|:----------------------------------------------------|:----------------------------------------------------|\n| 经理级及以上人员 | 一类城市                                            | 220                                                 | 50                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 25                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 经理级及以上人员 | 二类城市                                            | 200                                                 | 35                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 15                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 经理级及以上人员 | 三类城市                                            | 180                                                 | 30                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 15                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 其他工作人员     | 一类城市                                            | 180                                                 | 35                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 20                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 其他工作人员     | 二类城市                                            | 160                                                 | 30                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 15                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 其他工作人员     | 三类城市                                            | 150                                                 | 25                                                  | 汽车、火车(车程在5个小时以内的为硬座,以上为硬卧)    | 10                                                  | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 说明             | 一类城市指:北京、上海、杭州、天津、重庆、福州、厦门 | 一类城市指:北京、上海、杭州、天津、重庆、福州、厦门 | 一类城市指:北京、上海、杭州、天津、重庆、福州、厦门 | 一类城市指:北京、上海、杭州、天津、重庆、福州、厦门 | 一类城市指:北京、上海、杭州、天津、重庆、福州、厦门 | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 说明             | 二类城市指:一类城市之外的省会城市和地级市           | 二类城市指:一类城市之外的省会城市和地级市           | 二类城市指:一类城市之外的省会城市和地级市           | 二类城市指:一类城市之外的省会城市和地级市           | 二类城市指:一类城市之外的省会城市和地级市           | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n| 说明             | 三类城市指:县级市、县及以下级别城市                 | 三类城市指:县级市、县及以下级别城市                 | 三类城市指:县级市、县及以下级别城市                 | 三类城市指:县级市、县及以下级别城市                 | 三类城市指:县级市、县及以下级别城市                 | 超出规定以外的须申报总经理同意,否则超出费用自行承担 |\n第八条 出差住宿费报销审核1、住宿费按照上表实行包干制,超出部门需自行承担解决:2、因外事或重要业务需要招待宾客住宿或需要提高宾客及随同人员住宿费标准的,应事先向部门主管报告。经审核同意后执行,报销时须附注事由说明。3、总经理级(含)以上管理人员出差,可住单人间。其他同性员工两人一起出差的,应同住一间房,报销额不超过两人报销标准的合计额20%。4、出差期间有下列情况之一者,不予报销住宿费:⑴ 由接待单位免费接待安置的;⑵ 参加会议、学习、培训,其会议培训期间支付的会议费、培训费、资料服务费等综合性费用中已包括了住宿费的。第九条 餐饮补贴报销审核1、餐饮补助费用报销额按出差期间应就餐天数计算确定,实行包干制。不满一天按半天报销。2、出差期间因公发生对外业务招待费的按公司业务招待管理办法执行。但陪同招待就餐人员,按陪餐次数减发餐饮补贴。3、出差期间有下列情况之一者,不计发餐饮补贴:(1) 派驻外地工作人员回到家庭居住地办理公务期间;(2) 借出差之际绕道探亲、访友的期间;(3)支付会务费、培训费、咨询服务费等综合性费用中已包括了膳食费的。第十条 交通费报销审核1、出差乘坐飞机、火车、轮船或长途汽车应按不超过规定的等级执行,并凭出差地点、路线、时间一致的有效票据报销。2、在出差地办理公务以乘坐公共交通工具为原则,任务紧急或携带公物重的可选择乘坐计程车。市(县、区)内交通费报销时按规定附有效报销凭证(部门职员需经得部门主管同意方可、主管以上人员需经总经理同意方可)。3、多人一起出差,同乘一辆计程车,其交通费只能由一个持票人代为报销,同行其他人员不得再报销。4、经批准同意,个人或多个人一起同行出差自带(驾)私车,凭票报销与出差时间、出差地点一致的过路费。燃油费按行驶公里数0.9元/公里报销,随同人员不再报销相关交通费。5、出差期间有下列情况之一者,不予报销交通费:(1)未经同意,擅自乘坐超标准而发生的超支部分;(2)借出差之际探亲、访友和旅游而发生的交通费增支部分;(3)出差有公司商务车或搭乘公司员工车辆的;(4)未经批准,自带私车发生费用超过乘坐公共交通工具费用的部分。第十一条 出差期间杂费报销审核1、出差期间因业务所发生的电传费、邮递费、文件资料及公物托运费等,凭有效票据按实报销。2、员工因工作地点调动,随行行李托运费,凭有效凭证按实报销。第十二条 出差期间公休与延期的处理1、出差中公休:出差过程中,若遇公休日,应以公务为重,继续执行出差任务,如遇到出差单位公休无法办公,可安排休息;为避免此类情况,出差前应事先与有关单位联系。2、超期停留:出差过程中,因生病(未住院)、交通中断或其他不可抗拒因素造成超过预定期限停留的,在核清事实的基础上,按照实际差旅费报销;因生病(需住院)原因造成的意外,按标准支付出差补贴,不报销住宿费和市内交通费;住院期间出差补贴不超过5天;超期停留报销时需附上情况说明并经总经理同意后方可。3、出差期间不能按时回公司时,出差者必须向主管领导报告公务进展情况,并保持24小时开机。第十三条 差旅费报销程序1、出差人员回公司后,7个工作日内应按规定办理报销手续、结清借款。对无故未报销者不予再次借款。超过报销期,账面有个人出差借款的,财务部有权从当月工资中先行扣回。'
"""

In [12]:
q = "经理在一类城市的住宿费、餐费、交通费标准是多少"
prompt_test = f"""请根据以下公司规定回答问题。如果没有包含相关信息，则回复“没有相关信息”。
{tchunk}
问题：{q}
"""

answer = post_api_for_llm(prompt_test)
print(
    "问题\t",q,
    "对应的答案:\t",answer
)
print("-" * 50)

KeyboardInterrupt: 